In [1]:
# baseline cnn model
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from os import listdir
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from PIL import Image
import numpy as np
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [108]:
# load train and test dataset
def load_dataset():
    folder = "C:/Users/Dani/data/images/"
    onlyfiles = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    
    train_files = []
    y_train = []
    i=0
    for _file in onlyfiles:
        train_files.append(_file)

    image_width = 1000
    image_height = 800
    ratio = 2

    image_width = int(image_width / ratio)
    image_height = int(image_height / ratio)

    channels = 3
    nb_classes = 1

    dataset = np.ndarray(shape=(len(train_files), channels, image_height, image_width),
                         dtype=np.float32)
    i = 0
    for _file in train_files:
        img = load_img(folder + "/" + _file)  # this is a PIL image
        img.thumbnail((image_width, image_height))
        # Convert to Numpy Array
        x = img_to_array(img)  
        x = x.reshape(-1, 3,400,500)
        # Normalize
#         x = (x - 128.0) / 128.0
        dataset[i] = x
        i += 1
        if i % 2 == 0:
            print("%d images to array" % i)
    print("All images to array!")

    for i in dataset:
        print(i.size)
    
    z_train, y_train = np.loadtxt('C:/Users/Dani/labeled.txt', delimiter=' | ', unpack = True)
    
    from sklearn.model_selection import train_test_split

    #Splitting 
    X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=0.4, random_state=0)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=0)
    print("Train set size: {0}, Val set size: {1}, Test set size: {2}".format(len(X_train), len(X_val), len(X_test)))
    
    print(y_train)
    print(y_test)
    print(y_val)
    
    return X_train, y_train, X_test, y_test

In [109]:
# test_array = np.random.rand(3,2)
# test_vector = np.random.rand(4)

# np.savez_compressed('C:/Users/Dani/data/images', a = test_array, b = test_vector)


In [110]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(3, 400, 500)))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same', data_format=None))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
# 	compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [111]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [114]:
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
# 	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=1, batch_size=32, verbose=0)
	# save model
	model.save('final_model.h5')

In [115]:
# entry point, run the test harness
run_test_harness()

2 images to array
4 images to array
6 images to array
8 images to array
10 images to array
12 images to array
All images to array!
600000
600000
600000
600000
600000
600000
600000
600000
600000
600000
600000
600000
Train set size: 7, Val set size: 3, Test set size: 2
[0. 0. 1. 1. 1. 1. 0.]
[0. 1.]
[0. 1. 1.]


ValueError: Error when checking target: expected dense_6 to have shape (10,) but got array with shape (1,)